In [5]:
# Copy pld files over from CustomMapData dir.
!mv ~/home/Documents/Warcraft\ III/CustomMapData/Chlorine/* raw/ 2>/dev/null || echo "Okay"
!ls -ltr raw/ | tail -n 5

Okay
-rwxrwxrwx 1 b b 14748 Aug  6 00:16 chlorine-stats-Loam-Dank-Joust-173-2.pld
-rwxrwxrwx 1 b b  9136 Aug  6 13:32 chlorine-stats-Fjord-Quirk-Pluck-717-2.pld
-rwxrwxrwx 1 b b 14160 Aug  7 00:07 chlorine-stats-Upend-Molt-Fjord-135-2.pld
-rwxrwxrwx 1 b b  5907 Aug  7 22:09 chlorine-stats-Scry-Droit-Urge-438-2.pld
-rwxrwxrwx 1 b b  7933 Aug  7 22:43 chlorine-stats-Keen-Molt-Jaunt-376-2.pld


In [6]:
# Load file content into memory.
from glob import glob
from pathlib import Path

file_contents = {}

for file in glob("raw/*.pld"):
    pfile = Path(file)
    with open(pfile) as f:
        file_contents[pfile.name] = f.read()

sample = file_contents[[file for file in list(file_contents.keys()) if "Molt-Jaunt-" in file][0]]
print(sample[:400] + "...")

function PreloadFiles takes nothing returns nothing

	call PreloadStart()
	call Preload( "")
//v0PT02p2,Cokemonkey11#1442PT02p3,Footman16#2933PT02p4,Onlineritter#2732PT02p5,Arkhes#21394PT6S1p3,Night HowlerPT6S8p3,Starter CircletPT20S8p3,SocksPT30S5p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0," )
	call Preload( "")
//0p8,0,0p9,0,0PT36S1p5,Steel ElementalPT36S8p5,Starter CircletPT49S1p2,Goblin Blo...


In [16]:
# Parse the exfiltrated data out from PLD.
def clean(file_content: str) -> str:
    valid_lines = [line.strip() for line in file_content.split("\n") if line.startswith("//")]
    cleaned_lines = [line[2:len(line) - 3] for line in valid_lines]
    return "".join(cleaned_lines)

print(clean(sample)[:400] + "...")

v0PT02p2,Cokemonkey11#1442PT02p3,Footman16#2933PT02p4,Onlineritter#2732PT02p5,Arkhes#21394PT6S1p3,Night HowlerPT6S8p3,Starter CircletPT20S8p3,SocksPT30S5p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0PT36S1p5,Steel ElementalPT36S8p5,Starter CircletPT49S1p2,Goblin BloodbomberPT49S8p2,Starter CircletPT1M0S6p4,0,0,0,0,p2,0,0,0,0,p5,0,0,0,0,p3,0,0,0,0PT1M0S5p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p...


In [17]:
# Parse all the events out from the sample.
from dataclasses import dataclass
import datetime
import isodate
import warnings

SchemaVersion = int

EVENT_TYPE = {
    0: "None",
    1: "ChooseHero",
    2: "Identify",
    3: "Win",
    4: "TowerDeath",
    5: "DamageSnapshot",
    6: "StatSnapshot",
    7: "PlayerLeave",
    8: "GetItem",
    9: "SanctumDeath",
}

@dataclass
class Event:
    timestamp: datetime.timedelta
    event_type: int
    player: int
    context: list[str]

def parse_damage_snapshot(chunk: str) -> Event:
    timestamp = chunk.split("S5")[0] + "S"
    context = chunk.split("S5")[1]

    return Event(timestamp=isodate.parse_duration("PT" + timestamp), event_type=5, player=-1, context=[context])

def parse_stat_snapshot(chunk: str) -> Event:
    timestamp = chunk.split("S6")[0] + "S"
    context = chunk.split("S6")[1]

    return Event(timestamp=isodate.parse_duration("PT" + timestamp), event_type=6, player=-1, context=[context])

def parse(code: str) -> tuple[SchemaVersion, list[Event]]:
    chunks = code.split("PT") # TODO: use a streaming parser.

    schema_version = int(chunks[0][1:])

    events = []

    for chunk in chunks[1:]:
        if "S5" in chunk:
            events.append(parse_damage_snapshot(chunk))
            continue

        if "S6" in chunk:
            events.append(parse_stat_snapshot(chunk))
            continue

        timestamp_split = chunk.split("p")

        if not "S" in timestamp_split[0] and timestamp_split[0][0] == "0":
            timestamp_split[0] = timestamp_split[0][0] + "S" + timestamp_split[0][1:]

        if len(timestamp_split) == 1:
            warnings.warn(f"Skipped {chunk} as malformed")
            continue

        timestamp = timestamp_split[0].split("S")[0] + "S"

        event_type = timestamp_split[0].split("S")[1]

        other = "".join(timestamp_split[1:])

        if "S" in timestamp and not timestamp.endswith("S") and len(timestamp.split("S")) == 2:
            timestamp_reproc = timestamp.split("S")
            timestamp = timestamp_reproc[0] + "S"

            other = timestamp_reproc[1] + "," + other

        player_split = other.split(",")

        player = player_split[0]

        context = player_split[1:]

        timestamp_suffix = "" if timestamp.endswith("S") else "S"

        if event_type == '5':
            print(chunk)

        events.append(Event(timestamp=isodate.parse_duration("PT" + timestamp + timestamp_suffix), event_type=int(event_type), player=int(player), context=context))

    return (schema_version, events)

schema_version, events = parse(clean(sample))
# print(f"Schema version: {schema_version}")

print("\n".join([str(event) for event in events])[:800] + "...")


Event(timestamp=datetime.timedelta(0), event_type=2, player=2, context=['Cokemonkey11#1442'])
Event(timestamp=datetime.timedelta(0), event_type=2, player=3, context=['Footman16#2933'])
Event(timestamp=datetime.timedelta(0), event_type=2, player=4, context=['Onlineritter#2732'])
Event(timestamp=datetime.timedelta(0), event_type=2, player=5, context=['Arkhes#21394'])
Event(timestamp=datetime.timedelta(seconds=6), event_type=1, player=3, context=['Night Howler'])
Event(timestamp=datetime.timedelta(seconds=6), event_type=8, player=3, context=['Starter Circlet'])
Event(timestamp=datetime.timedelta(seconds=20), event_type=8, player=3, context=['Socks'])
Event(timestamp=datetime.timedelta(seconds=30), event_type=5, player=-1, context=['p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0'...


In [19]:
print([str(event.context) for event in events[:10]])

["['Cokemonkey11#1442']", "['Footman16#2933']", "['Onlineritter#2732']", "['Arkhes#21394']", "['Night Howler']", "['Starter Circlet']", "['Socks']", "['p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0']", "['Steel Elemental']", "['Starter Circlet']"]


In [20]:
# Save off the parsed events as json.
import json

with open("cleaned.json", "w") as f:
    f.write(json.dumps([event.__dict__ for event in events], default=str))

In [32]:
# Player identity mapping
# TODO: something is broken for name-processing when name contains "p" i.e. "lep"
player_identity_events = [event for event in events if event.event_type == 2]
# [print(player_identity_event) for player_identity_event in player_identity_events]
player_identities = {identity_event.player: identity_event.context[0].split('#')[0] for identity_event in player_identity_events}
[print(player_identity) for player_identity in list(player_identities.items())]

# Player hero choice mapping
hero_choice_events = [event for event in events if event.event_type == 1]
# print(hero_choice_events[0])
player_heroes = {hero_event.player: hero_event.context[0] for hero_event in hero_choice_events}
[print(player_hero) for player_hero in list(player_heroes.items())]

(2, 'Cokemonkey11')
(3, 'Footman16')
(4, 'Onlineritter')
(5, 'Arkhes')
(3, 'Night Howler')
(5, 'Steel Elemental')
(2, 'Goblin Bloodbomber')
(4, 'Steel Elemental')


[None, None, None, None]

In [37]:
# Damage and hero damage graphs
import matplotlib.pyplot as pyplot
import pandas
import seaborn

damage_snapshot_events = [event for event in events if event.event_type == 5]
print(damage_snapshot_events[0].context[0])

def parse_damage_snapshot_context(context: str) -> dict[int, tuple[int, int]]:
    by_player = [cxt for cxt in context.split('p') if len(cxt)]
    ret = {}
    for triple in by_player:
        ret[triple.split(',')[0]] = (triple.split(',')[1], triple.split(',')[2])

    return ret

print(parse_damage_snapshot_context(damage_snapshot_events[0].context[0]))

data ={'time': [], 'player': [], 'damage': [], 'hero_damage': []}

for damage_event_snapshot in damage_snapshot_events:
    for player, (damage, hero_damage) in parse_damage_snapshot_context(damage_event_snapshot.context[0]).items():
        if (int(damage) > 0 or int(hero_damage) > 0) and (int(player) > 1):
            player_name = f"{int(player) + 1} {player_identities.get(int(player))} ({player_heroes.get(int(player))})"
            data['time'].append(damage_event_snapshot.timestamp.total_seconds() / 60.0)
            data['player'].append(player_name)
            data['damage'].append(int(damage))
            data['hero_damage'].append(int(hero_damage))

df = pandas.DataFrame(data)

seaborn.set_theme(style="dark", palette='pastel', context='notebook')

seaborn.lineplot(data=df, x='time', y='damage', hue='player', marker='o')
pyplot.title('Cumulative damage over time by player')
pyplot.xlabel('Time (minutes)')
pyplot.ylabel('Cumulative damage')
pyplot.savefig('damage.png', format='png', dpi=200)
pyplot.close()

# Hero damage
seaborn.lineplot(data=df, x='time', y='hero_damage', hue='player', marker='o')
pyplot.title('Damage dealt to enemy heroes, cumulative over time by player')
pyplot.xlabel('Time (minutes)')
pyplot.ylabel('Cumulative hero damage')
pyplot.savefig('hero_damage.png', format='png', dpi=200)
pyplot.close()

p0,0,0p1,0,0p2,0,0p3,0,0p4,0,0p5,0,0p6,0,0p7,0,0p8,0,0p9,0,0
{'0': ('0', '0'), '1': ('0', '0'), '2': ('0', '0'), '3': ('0', '0'), '4': ('0', '0'), '5': ('0', '0'), '6': ('0', '0'), '7': ('0', '0'), '8': ('0', '0'), '9': ('0', '0')}


In [40]:
# Tower and sanctum death events
tower_death_events = [event for event in events if event.event_type == 4]
# print(tower_death_events[0])

tower_deaths = [[], []]  # Which timestamps did towers die for players 0, 1?

for event in tower_death_events:
    tower_deaths[event.player].append(event.timestamp)

print(tower_deaths)

sanctum_kills = [event for event in events if event.event_type == 9]

sanctum_deaths = [[], []]  # Which timestamps did sanctums die for players 0, 1?

for event in sanctum_kills:
    sanctum_deaths[event.player].append(event.timestamp)

print(sanctum_deaths)

[[datetime.timedelta(seconds=969)], [datetime.timedelta(seconds=713), datetime.timedelta(seconds=796), datetime.timedelta(seconds=948), datetime.timedelta(seconds=1010), datetime.timedelta(seconds=1055), datetime.timedelta(seconds=1111), datetime.timedelta(seconds=1167), datetime.timedelta(seconds=1267)]]
[[], [datetime.timedelta(seconds=1308)]]


In [44]:
# Resource/gold graph

stat_snapshot_events = [event for event in events if event.event_type == 6]
print(stat_snapshot_events[0].context[0])

def parse_stat_snapshot_context(context: str) -> dict[int, tuple[int, int, int, int]]:
    by_player = [cxt for cxt in context.split('p') if len(cxt)]
    ret = {}
    for triple in by_player:
        ret[triple.split(',')[0]] = (triple.split(',')[1], triple.split(',')[2], triple.split(',')[3], triple.split(',')[4])

    return ret

print(parse_stat_snapshot_context(stat_snapshot_events[0].context[0]))

data ={'time': [], 'player_id': [], 'player': [], 'kills': [], 'assists': [], 'deaths': [], 'cs': [], 'gold_earned': []}

for stat_event_snapshot in stat_snapshot_events:
    for player, (kills, assists, deaths, cs) in parse_stat_snapshot_context(stat_event_snapshot.context[0]).items():
        if (int(kills) > 0 or int(assists) > 0 or int(deaths) > 0 or int(cs) > 0) and (int(player) > 1):
            time = stat_event_snapshot.timestamp
            tower_deaths_so_far = len([death for death in tower_deaths[1 - (int(player) % 2)] if death < time])

            player_name = f"{int(player) + 1} {player_identities.get(int(player))} ({player_heroes.get(int(player))})"
            data['time'].append(stat_event_snapshot.timestamp.total_seconds() / 60.0)
            data['player_id'].append(int(player))
            data['player'].append(player_name)
            data['kills'].append(int(kills))
            data['assists'].append(int(assists))
            data['deaths'].append(int(deaths))
            data['cs'].append(int(cs))
            data['gold_earned'].append(int(kills) * 10 + int(assists) * 5 + int(cs) + 10 * tower_deaths_so_far)

df = pandas.DataFrame(data)

unique_players = df['player_id'].unique()

colors = {
    player_id: seaborn.color_palette("Reds", n_colors=len(unique_players))[i//2]
    if player_id % 2 == 0
    else seaborn.color_palette("Blues", n_colors=len(unique_players))[i//2]
    for i, player_id in enumerate(sorted(unique_players))
}

name_to_color = {name: colors[id] for name, id in zip(df['player'], df['player_id'])}

seaborn.set_theme(style="dark", palette='pastel', context='notebook')

seaborn.lineplot(data=df, x='time', y='gold_earned', hue='player', palette=name_to_color, marker='o')
pyplot.title('Cumulative gold (earned) over time by player')
pyplot.xlabel('Time (minutes)')
pyplot.ylabel('Cumulative gold (earned)')

for time in tower_deaths[0]:
    pyplot.axvline(x=time.total_seconds() / 60.0, color='red', linestyle='--', linewidth=0.5)
    pyplot.text(time.total_seconds() / 60.0, 1, 'West tower death', color='red', horizontalalignment='right', fontsize=4, rotation=15)

for time in tower_deaths[1]:
    pyplot.axvline(x=time.total_seconds() / 60.0, color='blue', linestyle='--', linewidth=0.5)
    pyplot.text(time.total_seconds() / 60.0, 1, 'East tower death', color='blue', horizontalalignment='right', fontsize=4, rotation=15)

for time in sanctum_deaths[0]:
    pyplot.axvline(x=time.total_seconds() / 60.0, color='red', linestyle='--', linewidth=0.5)
    pyplot.text(time.total_seconds() / 60.0, 1, 'West sanctum death', color='red', horizontalalignment='right', fontsize=4, rotation=15)

for time in sanctum_deaths[1]:
    pyplot.axvline(x=time.total_seconds() / 60.0, color='blue', linestyle='--', linewidth=0.5)
    pyplot.text(time.total_seconds() / 60.0, 1, 'East sanctum death', color='blue', horizontalalignment='right', fontsize=4, rotation=15)

victories = [event for event in events if event.event_type == 3]
if len(victories):
    redblue = 'red' if victories[0].player == 0 else 'blue'
    westeast = 'West' if victories[0].player == 0 else 'East'
    pyplot.axvline(x=victories[0].timestamp.total_seconds() / 60.0, color=redblue, linestyle='--', linewidth=0.5)
    pyplot.text(victories[0].timestamp.total_seconds() / 60.0, 1, f'{westeast} victory', color='black', horizontalalignment='right', fontsize=4, rotation=90)

pyplot.savefig('gold.png', format='png', dpi=200)
pyplot.close()

p4,0,0,0,0,p2,0,0,0,0,p5,0,0,0,0,p3,0,0,0,0
{'4': ('0', '0', '0', '0'), '2': ('0', '0', '0', '0'), '5': ('0', '0', '0', '0'), '3': ('0', '0', '0', '0')}
